In [3]:
#packages
import numpy
import pandas as pd
import os
from riotwatcher import LolWatcher, ApiError

In [7]:
#need to put this into a class
lol_watcher = LolWatcher('RGAPI-cfee8d23-cb0b-4865-8102-624e4f91d0c9') #api key

In [170]:
def get_Player(region, name):
    valid_region_names = ['oc1', 'na1', 'eun1']
    if region in valid_region_names:
        player = lol_watcher.summoner.by_name(region, name.lower())
        return lol_watcher.league.by_summoner('oc1', player['id'])
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')    
        
def get_CurrentPatchVersions(region):
    valid_region_names = ['oce', 'na1', 'eune']
    if region in valid_region_names:
        return lol_watcher.data_dragon.versions_for_region(region)
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')   

def get_AllPatchVersions():
    return [i for i in lol_watcher.data_dragon.versions_all() if not i.startswith('lolpatch')]

def get_Champions(patchversion = 'latest'):
    if patchversion == 'latest':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in [get_AllPatchVersions()[0]]] #nested by patch
        return [val for sublist in champions for val in sublist] #compress into single list
    elif patchversion == 'all':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in get_AllPatchVersions()] #nested by patch
        return [val for sublist in champions for val in sublist] #compress into single list
    else:
        champions = [[(patch, champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in patchversion] #nested by patch
        return [val for sublist in champions for val in sublist] #compress into single list

def get_ChampionStats(champions):
        return [{'patch': patchversion, 'champion': champion, **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['stats']} for patchversion, champion in champions]

In [26]:
get_Player('oc1', 'xcreedthoughtsx')

{'id': 'SEQSoF72wi8iTaOpA9nzbbTHYr18_IChWRPi5aaM4xS6InI',
 'accountId': 'x9Qkcg-r5nUX_taS5_gMS2kGHVZEzXWPHbuNAm3xxgnLy7zUv1gO1846',
 'puuid': 'fpJpwXtRY-RMkbrJYfosY2W9PmmmbiVI0WTNsdbTbTEYN2_awMjQDKPQUkr75nVWpobn5wl1voPLvg',
 'name': 'xCreedThoughtsx',
 'profileIconId': 4903,
 'revisionDate': 1668673245000,
 'summonerLevel': 141}

In [23]:
get_CurrentPatchVersions('oce')

{'n': {'item': '12.22.1',
  'rune': '7.23.1',
  'mastery': '7.23.1',
  'summoner': '12.22.1',
  'champion': '12.22.1',
  'profileicon': '12.22.1',
  'map': '12.22.1',
  'language': '12.22.1',
  'sticker': '12.22.1'},
 'v': '12.22.1',
 'l': 'en_AU',
 'cdn': 'https://ddragon.leagueoflegends.com/cdn',
 'dd': '12.22.1',
 'lg': '12.22.1',
 'css': '12.22.1',
 'profileiconmax': 28,
 'store': None}

In [28]:
lol_watcher.league.by_summoner('oc1', 'SEQSoF72wi8iTaOpA9nzbbTHYr18_IChWRPi5aaM4xS6InI')

[{'leagueId': 'a34f4d0f-6a66-49a9-8aef-6a64d9546c3b',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'BRONZE',
  'rank': 'IV',
  'summonerId': 'SEQSoF72wi8iTaOpA9nzbbTHYr18_IChWRPi5aaM4xS6InI',
  'summonerName': 'xCreedThoughtsx',
  'leaguePoints': 0,
  'wins': 16,
  'losses': 27,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [29]:
lol_watcher.data_dragon.versions_for_region('oce')

{'n': {'item': '12.22.1',
  'rune': '7.23.1',
  'mastery': '7.23.1',
  'summoner': '12.22.1',
  'champion': '12.22.1',
  'profileicon': '12.22.1',
  'map': '12.22.1',
  'language': '12.22.1',
  'sticker': '12.22.1'},
 'v': '12.22.1',
 'l': 'en_AU',
 'cdn': 'https://ddragon.leagueoflegends.com/cdn',
 'dd': '12.22.1',
 'lg': '12.22.1',
 'css': '12.22.1',
 'profileiconmax': 28,
 'store': None}

In [36]:
[champion]['stats']

{'hp': 650,
 'hpperlevel': 114,
 'mp': 0,
 'mpperlevel': 0,
 'movespeed': 345,
 'armor': 38,
 'armorperlevel': 4.45,
 'spellblock': 32,
 'spellblockperlevel': 2.05,
 'attackrange': 175,
 'hpregen': 3,
 'hpregenperlevel': 1,
 'mpregen': 0,
 'mpregenperlevel': 0,
 'crit': 0,
 'critperlevel': 0,
 'attackdamage': 60,
 'attackdamageperlevel': 5,
 'attackspeedperlevel': 2.5,
 'attackspeed': 0.651}

In [104]:
patchversion = '12.22.1'
champion = 'Aatrox'

lol_watcher.data_dragon.champions(patchversion)['data'][champion]['stats']

{'hp': 650,
 'hpperlevel': 114,
 'mp': 0,
 'mpperlevel': 0,
 'movespeed': 345,
 'armor': 38,
 'armorperlevel': 4.45,
 'spellblock': 32,
 'spellblockperlevel': 2.05,
 'attackrange': 175,
 'hpregen': 3,
 'hpregenperlevel': 1,
 'mpregen': 0,
 'mpregenperlevel': 0,
 'crit': 0,
 'critperlevel': 0,
 'attackdamage': 60,
 'attackdamageperlevel': 5,
 'attackspeedperlevel': 2.5,
 'attackspeed': 0.651}

In [176]:
champions = [get_Champions('latest')[0]]

In [205]:
champions = [
    ('12.22.1', 'Aatrox'),
    ('12.21.1', 'Aatrox'),
    ('12.22.1', 'Ahri'),
    ('12.21.1', 'Ahri')
]

In [206]:
data = [{'patch': patchversion, 'champion': champion, **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['stats']} for patchversion, champion in champions]

In [209]:
pd.DataFrame.from_dict(data)

,patch,champion,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,12.22.1,Aatrox,650,114,0,0,345,38,4.45,32,...,3.0,1.0,0,0.0,0,0,60,5,2.5,0.651
1,12.21.1,Aatrox,650,114,0,0,345,38,4.45,32,...,3.0,1.0,0,0.0,0,0,60,5,2.5,0.651
2,12.22.1,Ahri,570,96,418,25,330,18,4.70,30,...,2.5,0.6,8,0.8,0,0,53,3,2.0,0.668
3,12.21.1,Ahri,570,96,418,25,330,18,4.70,30,...,2.5,0.6,8,0.8,0,0,53,3,2.0,0.668


In [211]:
pd.DataFrame(get_Champions()) #make get_champions output a dictionary so that you can create data frame - carefull as this will effect get_championstats

,0,1
0,12.22.1,Aatrox
1,12.22.1,Ahri
2,12.22.1,Akali
3,12.22.1,Akshan
4,12.22.1,Alistar
...,...,...
157,12.22.1,Zeri
158,12.22.1,Ziggs
159,12.22.1,Zilean
160,12.22.1,Zoe
